In [8]:
import re
import os
import json
import pickle
import random
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from collections import OrderedDict
from bs4 import BeautifulSoup, element

from transformer.utils.tokenizer import MecabTokenizer, SpmTokenizer
from transformer.data.dataset import DatasetInterface, DatasetFromDir
from transformer.preprocessors.bert_preprocessor import BertPreprocessor
from transformer.preprocessors.blender_bot_preprocessor import GeneratorPretrainingPreprocessor
from transformer.preprocessors.utils import split_segment_by_speaker_ids, convert_turn_ids, flatten_sequence
from transformer.utils.common import get_nth_index

/Users/aibud_dev/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/cuda/amp/autocast_mode.py:118: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


In [7]:
A = "무슨 일로 오셨나요?"
B = "무슨 고민이 있어 오셨어요?"

In [21]:
token_ids = spm_tokenizer.tokens_to_ids(tokens=tokens_a, mask=False)
token_ids_b = spm_tokenizer.tokens_to_ids(tokens=tokens_b, mask=False)
subtokens_a = [spm_tokenizer.IdToPiece(token_id) for token_id in token_ids]
subtokens_a = [spm_tokenizer.IdToPiece(token_id) for token_id in token_ids]

NameError: name 'spm_tokenizer' is not defined

In [32]:
subtoken_pieces_a = preprocessor.src_spm_tokenizer.tokens_to_pieces(tokens=["test", "문장", "입니다."], mask=False)
# subtoken_pieces_b = spm_tokenizer.tokens_to_pieces(tokens=tokens_b, mask=False)

In [33]:
subtoken_pieces_a

['▁t', 'e', 'st', '▁문', '장', '▁입니다', '.']

In [6]:
0.0009 ** 1/128

7.03125e-06

### Load Dataset

In [14]:
# AIBUD_DEV
dataset_dir = "/Users/aibud_dev/_jupyter"
path = "./config/file_path.json"
file_path = None
with open(path, "r", encoding="utf-8") as fp:
    file_path = json.load(fp)

# # Picas_Server
# dataset_dir = "/home/picas/_jupyter"
# path = "./config/file_path.json"
# file_path = None
# with open(path, "r", encoding="utf-8") as fp:
#     file_path = json.load(fp)

# # Korea_Server
# dataset_dir = "/home/mnt/guest1"
# path = "./config/file_path.json"
# file_path = None
# with open(path, "r", encoding="utf-8") as fp:
#     file_path = json.load(fp)

# # bigshane_local
# dataset_dir = "D:\_jupyter"
# path = "./config/file_path.json"
# file_path = None
# with open(path, "r", encoding="utf-8") as fp:
#     file_path = json.load(fp)

In [15]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

def trim_obj(obj):
    obj = list(set(obj))
    obj = [e for e in obj if e!=""]
    obj = sorted(obj)
    return obj
    
def show_five_nums(data):
    quartiles = np.percentile(data, [25, 50, 75])
    min_v = np.min(data)
    max_v = np.max(data)
    avg = np.mean(data)
    print("Min: {min_v:.3f}\tMax: {max_v:.3f}\tAvg: {avg:.3f}\tQ1: {q1:.3f}\tQ2: {q2:.3f}\tQ3: {q3:.3f}".format(min_v=min_v, max_v=max_v, avg=avg, q1=quartiles[0], q2=quartiles[1], q3=quartiles[2]))
    return min_v, max_v, quartiles[0], quartiles[1], quartiles[2], avg



### load preprocessor

In [16]:
src_language = "kor"
tgt_language = "kor"
encoding = "utf-8"

src_vocab_size = tgt_vocab_size = 15000
embedding_dict = {
    "segment": 2, # context, condition
}
src_spm_model_path = dataset_dir + "/spm_model/{language}/spoken_pretrain_spm_v{vocab_size}".format(language=src_language, vocab_size=src_vocab_size)
tgt_spm_model_path = dataset_dir + "/spm_model/{language}/spoken_pretrain_spm_v{vocab_size}".format(language=tgt_language, vocab_size=tgt_vocab_size)
preprocessor = GeneratorPretrainingPreprocessor(src_language=src_language, tgt_language=tgt_language, src_spm_model_path=src_spm_model_path, tgt_spm_model_path=tgt_spm_model_path, embedding_dict=embedding_dict)
src_pad_token_id = preprocessor.src_spm_tokenizer.special_token_dict["pad"]["id"]
tgt_pad_token_id = preprocessor.tgt_spm_tokenizer.special_token_dict["pad"]["id"]

Imported konlpy.tag.Mecab successfully
loaded spm_model: '/Users/aibud_dev/_jupyter/spm_model/kor/spoken_pretrain_spm_v15000/'


## Wriet as Json format

In [4]:
# with open(root_dir + "/{language}/multi_turn/feed_data_0.json".format(language=language), "w", encoding=encoding) as fp:
#     json.dump(output, fp)

<br><br><br><hr><br><br><br>

## Make dialog pretrain data

In [354]:
dataset_name_list = ["KLUE_DST", "NIKL_2020", "NIKL_Dialog", "AIHUB_Twitter", "AIHUB_SSGI", "AIHUB_KETI", "AIHUB_SNS", "AIHUB_EMOTION", "KaggleConversation", "OpenSubtitles"]
output_dir = dataset_dir + "/dataset/preprocessed/dialog_pretrain/kor/multi_turn/"
# dataset_name_list = ["KLUE_DST", "AIHUB_Twitter", "AIHUB_SSGI", "AIHUB_KETI", "AIHUB_SNS", "AIHUB_EMOTION", "NIKL_2020", "NIKL_Dialog", "KaggleConversation"]
# output_dir = dataset_dir + "/dataset/preprocessed/dialog_pretrain/kor/multi_turn_v2/"
# dataset_name_list = ["KLUE_DST", "AIHUB_Twitter", "AIHUB_SSGI", "AIHUB_KETI", "AIHUB_SNS", "AIHUB_EMOTION"]
# output_dir = dataset_dir + "/dataset/preprocessed/dialog_pretrain/kor/multi_turn_v3/"

In [355]:
min_num_turns = 4
max_num_turns = 8
shuffle = True
train_ratio = 0.85
num_samples = 5000

language = "kor"
encoding = "UTF-8"
extension = "json"
print("# Load data from datasets...")
stat_dict = OrderedDict()
data = OrderedDict()

for dataset_name in dataset_name_list:
    root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
    if "multi_turn" not in file_path[dataset_name]["feed_data"]: continue

    data_path = file_path[dataset_name]["feed_data"]["multi_turn"].format(root_dir=root_dir, language=language)
    for file_name in os.listdir(data_path):
        if not file_name.endswith(extension): continue
        with open(data_path+file_name, "r", encoding=encoding) as fp:
            _data = json.load(fp)

            if dataset_name not in data: data[dataset_name] = []
            data[dataset_name] += _data
            _avg_utt_size = np.round(np.mean([len(r["utterances"]) for r in _data]))
            print("\t", dataset_name, ":", len(_data), "\t\tavg(utt_size):", _avg_utt_size)
            stat_dict[dataset_name] = dict()
            stat_dict[dataset_name]["dataset_size"] = len(_data)
            stat_dict[dataset_name]["avg_utterances"] = _avg_utt_size
print()

print("# Extract utterances from data...")
output = []
train_output = []
val_output = []

for dataset_name, _data in data.items():
    _train_rows = []
    _val_rows = []
    
    train_idx = int(len(_data) * train_ratio)
    for _row_idx, _row in enumerate(_data):
        _num_turns = np.random.randint(low=min_num_turns, high=max_num_turns)
        sequence_utterances, sequence_turn_ids = split_segment_by_speaker_ids(utterances=_row["utterances"], speaker_ids=_row["speaker_ids"])
        num_speakers = len(set([turn_id for segment_turn_id in sequence_turn_ids for turn_id in segment_turn_id]))
        if num_speakers != 2: continue

        turn_iter = max(len(sequence_utterances) - _num_turns, 0)
        for i in range(0, turn_iter+1): 
            _sequence_utterances = sequence_utterances[i:i+_num_turns]
            _sequence_turn_ids = sequence_turn_ids[i:i+_num_turns]
            _sequence_turn_ids = convert_turn_ids(_sequence_turn_ids, model_id=0, user_id=1)
            _utterances, _speaker_ids = flatten_sequence(sequence_utterances=_sequence_utterances, sequence_turn_ids=_sequence_turn_ids)               
            row_dict = dict()
            row_dict["utterances"] = _utterances
            row_dict["speaker_ids"] = _speaker_ids
            
            if _row_idx <= train_idx:
                _train_rows.append(row_dict)
            else:
                _val_rows.append(row_dict)

    train_output += _train_rows
    val_output += _val_rows
    print("\t{} : (train: {}, val: {})".format(dataset_name, len(_train_rows), len(_val_rows)), "(empty_row_cnt: {})".format(empty_row_cnt))
    stat_dict[dataset_name]["dialog_size"] = (len(_train_rows), len(_val_rows))
output = train_output + val_output
print()
    

output_filename_template = "feed_data_{idx}.json"
size_per_file = 100000

print("# Write extracted dialogs...")
if shuffle: random.shuffle(output)
if not os.path.exists(output_dir) or not os.path.isdir(output_dir): os.makedirs(output_dir)
for idx in range(0, len(output)//size_per_file+1):
    begin_idx = idx * size_per_file
    end_idx = min(len(output), (idx+1) * size_per_file)
    _output = output[begin_idx:end_idx]
    output_filename = output_filename_template.format(idx=str(idx).zfill(2))
    with open(output_dir+output_filename, "w", encoding="utf-8") as fp:
        print("\t{}: {}".format(output_filename, len(_output)))
        json.dump(_output, fp)

if not os.path.isdir(output_dir + "train"): os.mkdir(output_dir + "train")
for idx in range(0, len(train_output)//size_per_file+1):
    begin_idx = idx * size_per_file
    end_idx = min(len(train_output), (idx+1) * size_per_file)
    _output = train_output[begin_idx:end_idx]
    output_filename = output_filename_template.format(idx=str(idx).zfill(2))
    with open(output_dir+"train/"+output_filename, "w", encoding="utf-8") as fp:
        print("\ttrain {}: {}".format(output_filename, len(_output)))
        json.dump(_output, fp)
        
if not os.path.isdir(output_dir + "val"): os.mkdir(output_dir + "val")
for idx in range(0, len(val_output)//size_per_file+1):
    begin_idx = idx * size_per_file
    end_idx = min(len(val_output), (idx+1) * size_per_file)
    _output = val_output[begin_idx:end_idx]
    output_filename = output_filename_template.format(idx=str(idx).zfill(2))
    with open(output_dir+"val/"+output_filename, "w", encoding="utf-8") as fp:
        print("\tval {}: {}".format(output_filename, len(_output)))
        json.dump(_output, fp)
        
# write sample data
if not os.path.isdir(output_dir + "sample"): os.mkdir(output_dir + "sample")
sample_data_path = output_dir + "sample/feed_data_sample.json"
with open(sample_data_path, "w", encoding="utf-8") as fp:
    json.dump(output[:num_samples], fp)
    
# write DESC.md
with open(output_dir + "DESC.md", "w", encoding="utf-8") as fp:
    _str = "## DESC\n" +\
        "- Korean Dialog dataset for Language Model training\n" +\
        "- total: {size}, avg_turns: {avg:.3f}\n".format(size=len(output), avg=np.mean([len(row["utterances"]) for row in output])) +\
        "- min turns: {mn_t}, max turns: {mx_t}\n".format(mn_t=min_num_turns, mx_t=max_num_turns) +\
        "\n" +\
        "## dataset statistics\n" +\
        "### dataset_name: (avg_utterances, num_dialogues_before, num_dialogues_after)\n"
    for dataset_name, stat in stat_dict.items():
        _row_stat = "\t- {dataset_name}: ({avg_utterances}, {num_dialogues_before}, {num_dialogues_after})\n".format(dataset_name=dataset_name, avg_utterances=stat["dataset_size"], num_dialogues_before=stat["avg_utterances"], num_dialogues_after=sum(stat["dialog_size"]))
        _str += _row_stat
    fp.write(_str)

# Load data from datasets...
	 KLUE_DST : 9000 		avg(utt_size): 15.0
	 NIKL_2020 : 2232 		avg(utt_size): 66.0
	 NIKL_Dialog : 102 		avg(utt_size): 510.0
	 AIHUB_Twitter : 1999 		avg(utt_size): 7.0
	 AIHUB_SSGI : 5855 		avg(utt_size): 15.0
	 AIHUB_KETI : 4882 		avg(utt_size): 9.0
	 AIHUB_KETI : 5337 		avg(utt_size): 6.0
	 AIHUB_KETI : 749 		avg(utt_size): 7.0
	 AIHUB_SNS : 36680 		avg(utt_size): 11.0
	 AIHUB_EMOTION : 84212 		avg(utt_size): 5.0
	 KaggleConversation : 849 		avg(utt_size): 5.0
	 OpenSubtitles : 1023834 		avg(utt_size): 12.0

# Extract utterances from data...
	 KLUE_DST : 28272
	 NIKL_2020 : 6811
	 NIKL_Dialog : 13189
	 AIHUB_Twitter : 2900
	 AIHUB_SSGI : 17805
	 AIHUB_KETI : 12575
	 AIHUB_SNS : 35889
	 AIHUB_EMOTION : 84212
	 KaggleConversation : 849
	 OpenSubtitles : 1782125

# Write extracted dialogs...
	 feed_data_00.json : 100000
	 feed_data_01.json : 100000
	 feed_data_02.json : 100000
	 feed_data_03.json : 100000
	 feed_data_04.json : 100000
	 feed_data_05.json : 10

<br><br><br><hr><br><br><br>

## Make dialog finetuning data

In [20]:
host = "3.143.59.250"
port = "8000"
service = "dialog-retriever"

method = "infer-next-utterance"
body = {
    "utterances": [],
    "speaker_ids": [],
    "top_n": 5,
    "max_retry": 5
}

url = "http://{host}:{port}/{service}/{method}".format(host=host, port=port, service=service, method=method)

from transformer.utils.common import get_nth_index
def get_condition_from_remote_retriever(utterances, speaker_ids):
    last_user_idx = get_nth_index(obj=speaker_ids, value=1, n=-1)
    _utterances = utterances[0:last_user_idx+1]
    _speaker_ids = speaker_ids[0:last_user_idx+1]
    body["utterances"] = _utterances
    body["speaker_ids"] = _speaker_ids
    response = requests.post(url, json=body)
    response_json = response.json()
    
    condition = None
    if response_json["status"]:
        condition = [response_json["output"][0][0]]
    else:
        print("{}: {}".format(response_json["status"], response_json["error_message"]))
        condition = utterances[last_user_idx+1:]
    return condition

0.5323439392779867

In [15]:
from transformer.services.dialog_retriever.poly_encoder import DialogRetriever
_model_dir = "/home/mnt/guest1/model/poly_encoder/dialog_retriever/20210813/"
dr = DialogRetriever(temp_dir="./")
dr.load_model(model_dir=_model_dir)

def get_condition_from_retriever(utterances, speaker_ids, top_n=5, max_retry=5):
    condition = None
    try:
        output = dr.infer_next_utterance(utterances=utterances, speaker_ids=speaker_ids, top_n=top_n, max_retry=max_retry)
        condition = output[0][0]
    except:
        print("length over")
        condition = utterances[last_user_idx+1:]
    return condition

In [16]:
def get_condition(utterances, speaker_ids, personas, entities):
    user_persona = [persona for persona in personas if persona["id"]==1][0]
    is_user_condition = False
    if "name" in entities and len(entities["name"]) > 0:
        last_entity_info = entities["name"][-1]
        entity_row_idx, entity_begin_idx, entity_end_idx, entity_span = last_entity_info
        name_in_utterance = utterances[entity_row_idx][entity_begin_idx:entity_end_idx]
        if entity_row_idx + 1 == len(utterances) and name_in_utterance in user_persona["name"]: is_user_condition = True
    
    condition = None
    if is_user_condition:
        # 유저 정보를 이용하면, 컨디션으로 유저 정보를 넣어줘야지    
        user_name = user_persona["name"]
        name_condition = {"name": user_name}
        condition = [name_condition.__str__()]
#         condition = [user_persona.__str__()]
    else:
        condition = None
#         condition = get_condition_from_retriever(utterances=utterances, speaker_ids=speaker_ids)
    return condition

pd.options.mode.chained_assignment = None  # default='warn'
def find_most_similar_persona(age, gender, default_name="당신"):
    target_df = persona_df
    target_persona = {"name": default_name}
    try:
        target_df = target_df[target_df["gender"]==gender]
        age_diff = abs(target_df["age"] - age)
        target_df["age_diff"] = list(age_diff)
        target_age_diff = target_df.sort_values("age_diff").values.tolist()[0][-1]
        target_df = target_df.loc[(target_df["age_diff"]==target_age_diff)]
        target_persona = target_df.sample(1).to_dict("records")[0]
    except:
        target_persona = target_df.sample(1).to_dict("records")[0]
    return target_persona

def get_random_name():
    random_name = np.random.choice(persona_df["name"].unique().tolist())
    return random_name

# target_idx = 137
# _utterances = output[target_idx]["utterances"][:-1]
# _speaker_ids = output[target_idx]["speaker_ids"][:-1]
# for _utterance, _speaker_id in zip(_utterances, _speaker_ids):
#     print("{}: {}".format(_speaker_id, _utterance))

# body["utterances"] = _utterances
# body["speaker_ids"] = _speaker_ids
# response = requests.post(url, json=body)
# response.json()

In [17]:
dataset_name_list = ["SelectStar", "EmpatheticDialogues"]
# dataset_name_list = ["SelectStar"]
output_dir = dataset_dir + "/dataset/preprocessed/dialog_finetuning/kor/multi_turn/"
# output_dir = dataset_dir + "/dataset/preprocessed/dialog_finetuning/kor/multi_turn_v1/"

# multi_turn: 기본 데이터, strictly-shuffled (리트리버 컨디션 & 유저 컨디션)
# multi_turn_v1: 기본 데이터, lossely-shuffled (리트리버 컨디션 & 유저 컨디션)
# multi_turn_v2: user_data condtion only, strictly-shuffled (name 정보만)
# multi_turn_v3: user_data condtion only, lossely-shuffled (name 정보만)
# multi_turn_v2: user_data condtion only, augmented * 5, strictly-shuffled (name 정보만)
# multi_turn_v3: user_data condtion only, augmented * 5, lossely-shuffled (name 정보만)

# Load data from datasets...
	 SelectStar : 2342 		avg(utt_size): 31.0
	 EmpatheticDialogues : 618 		avg(utt_size): 18.0



In [106]:
min_num_turns = 2
max_num_turns = 6
shuffle = True
train_ratio = 0.85
num_samples = 5000
append_condition = True

language = "kor"
encoding = "UTF-8"
extension = "json"
print("# Load data from datasets...")
stat_dict = OrderedDict()
data = OrderedDict()

for dataset_name in dataset_name_list:
    root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
    if "multi_turn" not in file_path[dataset_name]["feed_data"]: continue

    data_path = file_path[dataset_name]["feed_data"]["multi_turn"].format(root_dir=root_dir, language=language)
    for file_name in os.listdir(data_path):
        if not file_name.endswith(extension): continue
        with open(data_path+file_name, "r", encoding=encoding) as fp:
            _data = json.load(fp)

            if dataset_name not in data: data[dataset_name] = []
            data[dataset_name] += _data
            _avg_utt_size = np.round(np.mean([len(r["utterances"]) for r in _data]))
            print("\t", dataset_name, ":", len(_data), "\t\tavg(utt_size):", _avg_utt_size)
            stat_dict[dataset_name] = dict()
            stat_dict[dataset_name]["dataset_size"] = len(_data)
            stat_dict[dataset_name]["avg_utterances"] = _avg_utt_size
print()

NameError: name 'row' is not defined

In [18]:
print("# Extract utterances from data...")
output = []
train_output = []
val_output = []

u_output = []
u_train_output = []
u_val_output = []

empty_row_cnt = 0
for dataset_name, _data in data.items():
    train_idx = int(len(_data) * train_ratio)
    
    _output = []
    _train_output = []
    _val_output = []
    _u_output = []
    _u_train_output = []
    _u_val_output = []
    for row_idx, row in enumerate(tqdm(_data, initial=0, total=len(_data), desc=dataset_name)): 
        if len([_utterance for _utterance in row["utterances"] if _utterance!=""]) < 1:
            empty_row_cnt += 1
            continue
        
        for begin_idx in range(0, (len(row["speaker_ids"]) - max_num_turns)):    
            for end_idx in range(begin_idx+1, len(row["speaker_ids"])):
                cur_num_turns = 0
                _prev_speaker_id = row["speaker_ids"][begin_idx]
                for _speaker_id in row["speaker_ids"][begin_idx+1:end_idx+1]:
                    if _speaker_id != _prev_speaker_id:
                        _prev_speaker_id = _speaker_id
                        if _speaker_id == 1: cur_num_turns += 1

                if begin_idx == 0 and cur_num_turns < 1: continue # 대화의 첫 발화이면 1턴 미만인 경우에 continue
                if begin_idx != 0 and cur_num_turns < min_num_turns: continue # 대화의 첫 발화가 아니면 min_num_turns턴 미만인 경우에 continue
                if cur_num_turns > max_num_turns: break # max_num_turns보다 크면 break
                if row["speaker_ids"][end_idx] != 0: continue # model_id로 발화가 끝나지 않는 경우 continue
                if len(row["speaker_ids"]) > end_idx+1 and row["speaker_ids"][end_idx] == row["speaker_ids"][end_idx+1]: continue # 다음 발화도 model_id의 것이면 continue

                _utterances = row["utterances"][begin_idx:end_idx+1]
                _speaker_ids = row["speaker_ids"][begin_idx:end_idx+1]
                _labels = row["labels"][begin_idx:end_idx+1]
                _persona = row["persona"]
                if isinstance(_persona, dict): _persona = list(_persona.values())
                _entities = dict()
                for k,v in row["entities"].items():
                    _entities[k] = []
                    for _v in v:
                        _entitiy_row_idx, _entitiy_begin_idx, _entitiy_end_idx, _entitiy_span = _v
                        entity_row_idx = _entitiy_row_idx - begin_idx
                        if entity_row_idx < 0: continue
                        if entity_row_idx > (end_idx - begin_idx): continue
                        _v_row = [entity_row_idx, _entitiy_begin_idx, _entitiy_end_idx, _entitiy_span]
                        _entities[k].append(_v_row)
                
                # utterance_length constraints: timestep보다 길면 탈락 (128 - num_special_tokens)
                concat_utterances = " ".join(_utterances)
                if preprocessor.get_src_token_length(sentence=concat_utterances) >= 125: continue
                    
                row_dict = row.copy()
                row_dict["utterances"] = _utterances
                row_dict["speaker_ids"] = _speaker_ids
                row_dict["labels"] = _labels
                row_dict["entities"] = _entities
                row_dict["persona"] = _persona
                _condition = get_condition(utterances=_utterances, speaker_ids=_speaker_ids, personas=_persona, entities=_entities)
                row_dict["condition"] = _condition

                _output.append(row_dict)
                u_reply = True if "name" in row_dict["entities"] and len(row_dict["entities"]["name"]) > 0 and row_dict["entities"]["name"][-1][0] == len(row_dict["utterances"])-1 else False
                if u_reply: _u_output.append(row_dict)
                if row_idx <= train_idx: 
                    _train_output.append(row_dict)
                    if u_reply: _u_train_output.append(row_dict)
                else: 
                    _val_output.append(row_dict)
                    if u_reply: _u_val_output.append(row_dict)
    
    output += _output
    train_output += _train_output
    val_output += _val_output
    u_output += _u_output
    u_train_output += _u_train_output
    u_val_output += _u_val_output
    print("\t{} : (train: {}, val: {})".format(dataset_name, len(_train_output), len(_val_output)), "(empty_row_cnt: {})".format(empty_row_cnt))
    stat_dict[dataset_name]["dialog_size"] = (len(_train_output), len(_val_output))
    
random.shuffle(train_output)
random.shuffle(val_output)
output = train_output + val_output

random.shuffle(u_train_output)
random.shuffle(u_val_output)
u_output = u_train_output + u_val_output

 63%|██████▎   | 1470/2342 [00:00<00:00, 7363.94it/s]

# Extract utterances from data...


100%|██████████| 618/618 [00:00<00:00, 13568.97it/s]

	SelectStar : (train: 179, val: 26) (empty_row_cnt: 1)
	EmpatheticDialogues : (train: 143, val: 72) (empty_row_cnt: 80)



In [19]:
output_filename_template = "feed_data_{idx}.json"
size_per_file = 100000

print("# Write extracted dialogs...")
if shuffle: random.shuffle(output)
if not os.path.exists(output_dir) or not os.path.isdir(output_dir): os.makedirs(output_dir)
for idx in range(0, len(output)//size_per_file+1):
    begin_idx = idx * size_per_file
    end_idx = min(len(output), (idx+1) * size_per_file)
    _output = output[begin_idx:end_idx]
    output_filename = output_filename_template.format(idx=str(idx).zfill(2))
    with open(output_dir+output_filename, "w", encoding="utf-8") as fp:
        print("\t{}: {}".format(output_filename, len(_output)))
        json.dump(_output, fp)

if not os.path.isdir(output_dir + "train"): os.mkdir(output_dir + "train")
for idx in range(0, len(train_output)//size_per_file+1):
    begin_idx = idx * size_per_file
    end_idx = min(len(train_output), (idx+1) * size_per_file)
    _output = train_output[begin_idx:end_idx]
    output_filename = output_filename_template.format(idx=str(idx).zfill(2))
    with open(output_dir+"train/"+output_filename, "w", encoding="utf-8") as fp:
        print("\ttrain {}: {}".format(output_filename, len(_output)))
        json.dump(_output, fp)
        
if not os.path.isdir(output_dir + "val"): os.mkdir(output_dir + "val")
for idx in range(0, len(val_output)//size_per_file+1):
    begin_idx = idx * size_per_file
    end_idx = min(len(val_output), (idx+1) * size_per_file)
    _output = val_output[begin_idx:end_idx]
    output_filename = output_filename_template.format(idx=str(idx).zfill(2))
    with open(output_dir+"val/"+output_filename, "w", encoding="utf-8") as fp:
        print("\tval {}: {}".format(output_filename, len(_output)))
        json.dump(_output, fp)
        
# write sample data
if not os.path.isdir(output_dir + "sample"): os.mkdir(output_dir + "sample")
sample_data_path = output_dir + "sample/feed_data_sample.json"
with open(sample_data_path, "w", encoding="utf-8") as fp:
    json.dump(output[:num_samples], fp)
    
# write DESC.md
with open(output_dir + "DESC.md", "w", encoding="utf-8") as fp:
    _str = "## DESC\n" +\
        "- Korean Dialog dataset for Language Model training\n" +\
        "- total: {size}, avg_turns: {avg:.3f}\n".format(size=len(output), avg=np.mean([len(row["utterances"]) for row in output])) +\
        "- min turns: {mn_t}, max turns: {mx_t}\n".format(mn_t=min_num_turns, mx_t=max_num_turns) +\
        "\n" +\
        "## dataset statistics\n" +\
        "### dataset_name: (avg_utterances, num_dialogues_before, num_dialogues_after)\n"
    for dataset_name, stat in stat_dict.items():
        _row_stat = "\t- {dataset_name}: ({avg_utterances}, {num_dialogues_before}, {num_dialogues_after})\n".format(dataset_name=dataset_name, avg_utterances=stat["dataset_size"], num_dialogues_before=stat["avg_utterances"], num_dialogues_after=sum(stat["dialog_size"]))
        _str += _row_stat
    fp.write(_str)

# Write extracted dialogs...
	feed_data_00.json: 420
	train feed_data_00.json: 322
	val feed_data_00.json: 98


In [ ]:
u_utterance = {
    "P": "즐거움",
    "J": "기쁨",
    "S": "슬픔",
    "A": "분노",
    "O": "보통"
}
g_utterance = {
    "U": "유저정보",
    "R": "페르소나",
    "K": "외부지식",
    "E": "공감형대화",
    "N": "일반/기타"
}

from collections import Counter
labels = [label for row in output for label in row["labels"]]
Counter(labels)

#### augment user_condition

In [ ]:
num_augment = 5
augmented_u_train_output = []
augmented_u_val_output = []

for row in u_train_output:
    augmented_u_train_output.append(row)
    
    model_persona = [persona for persona in row["persona"] if persona["id"]==0][0]
    user_persona = [persona for persona in row["persona"] if persona["id"]==1][0]
    for i in range(0, num_augment):
        _row = row.copy()
        new_name = get_random_name()
        new_persona = user_persona.copy()
        new_persona["name"] = new_name
        _persona = [model_persona, new_persona]
        _row["persona"] = _persona

        _utterances = _row["utterances"].copy()
        if "name" in _row["entities"]:
            name_to_change = new_name[1:] if len(new_name) == 3 else new_name
            name_entity = []
            for entity in _row["entities"]["name"]:
                _target_utterance = _utterances[entity[0]]
                new_utterance = _target_utterance[:entity[1]] + name_to_change + _target_utterance[entity[2]:]
                _utterances[entity[0]] = new_utterance
                _entity = entity.copy()
                _entity[2] = _entity[1] + len(name_to_change)
                _entity[3] = len(name_to_change)
                name_entity.append(_entity)
            _row["entities"]["name"] = name_entity
            _row["utterances"] = _utterances
        _condition = get_condition(utterances=_row["utterances"], speaker_ids=_row["speaker_ids"], personas=_row["persona"], entities=_row["entities"])
        if _condition is None:
            raise AssertionError
        else:
            _row["condition"] = _condition
            
        
        augmented_u_train_output.append(_row)
        
for row in u_val_output:
    augmented_u_val_output.append(row)
    
    model_persona = [persona for persona in row["persona"] if persona["id"]==0][0]
    user_persona = [persona for persona in row["persona"] if persona["id"]==1][0]
    for i in range(0, num_augment):
        _row = row.copy()
        new_name = get_random_name()
        new_persona = user_persona.copy()
        new_persona["name"] = new_name
        _persona = [model_persona, new_persona]
        _row["persona"] = _persona

        _utterances = _row["utterances"].copy()
        if "name" in _row["entities"]:
            name_to_change = new_name[1:] if len(new_name) == 3 else new_name
            name_entity = []
            for entity in _row["entities"]["name"]:
                _target_utterance = _utterances[entity[0]]
                new_utterance = _target_utterance[:entity[1]] + name_to_change + _target_utterance[entity[2]:]
                _utterances[entity[0]] = new_utterance
                _entity = entity.copy()
                _entity[2] = _entity[1] + len(name_to_change)
                _entity[3] = len(name_to_change)
                name_entity.append(_entity)
            _row["entities"]["name"] = name_entity
            _row["utterances"] = _utterances
        _condition = get_condition(utterances=_row["utterances"], speaker_ids=_row["speaker_ids"], personas=_row["persona"], entities=_row["entities"])
        if _condition is None:
            raise AssertionError
        else:
            _row["condition"] = _condition
        
        augmented_u_val_output.append(_row)
        
random.shuffle(augmented_u_train_output)
random.shuffle(augmented_u_val_output)

<br><br><br><hr><br><br><br>

## SelectStar

In [117]:
u_utterance = {
    "P": "즐거움",
    "J": "기쁨",
    "S": "슬픔",
    "A": "분노",
    "O": "보통"
}
g_utterance = {
    "U": "유저정보",
    "R": "페르소나",
    "K": "외부지식",
    "E": "공감형대화",
    "N": "일반/기타"
}

In [148]:
dataset_name = "SelectStar"
language = "kor"
encoding = "UTF-8"
extension = "json"

root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

data = []
for filename in os.listdir(raw_dir):
    if not filename.endswith(extension): continue
    with open(raw_dir+filename, "r", encoding=encoding) as fp:
        __data = json.load(fp)
        
        _data = []
        for row in __data:
            if "speakers_ids" in row:
                row["speaker_ids"] = row.pop("speakers_ids")
            _data.append(row)
        data += _data
print("data size:", len(data))

error_indice = []
for row_idx, row in enumerate(data):
    error_list = []
    if len(row["persona"])!=2: 
        error_list.append("persona length: {}".format(len(row["persona"])))
    if len(row["utterances"])!=len(row["speaker_ids"]) or len(row["utterances"])!=len(row["labels"]) or len(row["speaker_ids"])!=len(row["labels"]): 
        error_list.append("length difference: {}/{}/{}".format(len(row["utterances"]),len(row["speaker_ids"]),len(row["labels"])))

    for speaker_id, label in zip(row["speaker_ids"], row["labels"]):
        if speaker_id == 0 and label not in g_utterance:
            error_list.append("wrong label for g: '{}'".format(label))
        if speaker_id == 1 and label not in u_utterance:
            error_list.append("wrong label for u: '{}'".format(label))

    if len(error_list) > 0:
        error_indice.append((row_idx, error_list))
        print("error: {}".format(row_idx))
        
string_output = ""
for row in error_indice:
    _str = "{}:\n".format(row[0])
    for error in row[1]:
        _str += "\t{}\n".format(error)
    string_output += _str
    
print("error nums:", len(error_indice))
if len(error_indice) <= 0:
    random.shuffle(output)
    with open(root_dir + "/{language}/multi_turn/feed_data_0.json".format(language=language), "w", encoding=encoding) as fp:
        json.dump(data, fp)
else:
    with open("./error_contents.txt", "w", encoding="utf-8") as fp:
        fp.write(string_output)

data size: 2342
error nums: 0


## AIHUB_SSGI

In [ ]:
dataset_name = "AIHUB_SSGI"
language = "kor"
encoding = "UTF-8"
root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

final_output = []
file_name_list = [file_name for file_name in os.listdir(raw_dir+"dialog") if file_name.endswith(".xlsx")]
for file_name in file_name_list:
    df = pd.read_excel(raw_dir+"dialog/"+file_name, engine="openpyxl")
    df = df.fillna("")
    
    speaker_info = dict()
    domain = None
    utterances = []
    speaker_ids = []
    categories = []
    tasks = []
    subtasks = []

    idx = 0
    output = []
    for row in df[["SPEAKER", "SENTENCE", "DOMAIN", "CATEGORY", "MAIN", "SUB", "SPEAKERID", "SENTENCEID"]].values.tolist():
        speaker, sentence, _domain, category, main, sub, speaker_id, sentence_id = row

        if int(sentence_id) == 1 and len(utterances) > 0:
            # append
            output_row = OrderedDict()
            output_row["idx"] = idx
            output_row["speakers"] = dict(OrderedDict(sorted(speaker_info.items())))
            output_row["domain"] = domain
            output_row["utterances"] = utterances
            output_row["speaker_ids"] = speaker_ids
            output_row["category"] = trim_obj(categories)
            output_row["task"] = trim_obj(tasks)
            output_row["subtask"] = trim_obj(subtasks)
            output_row = dict(output_row)
            output.append(output_row)

            # reset
            idx += 1
            speaker_info = dict()
            domain = None
            utterances = []
            speaker_ids = []
            categories = []
            tasks = []
            subtasks = []

        speaker_info[speaker_id] = speaker
        domain = _domain
        utterances.append(sentence)
        speaker_ids.append(speaker_id)
        categories.append(category)
        tasks.append(main)
        subtasks.append(sub)

    # append
    output_row = OrderedDict()
    output_row["idx"] = idx
    output_row["speakers"] = speaker_info
    output_row["domain"] = domain
    output_row["utterances"] = utterances
    output_row["speaker_ids"] = speaker_ids
    output_row["category"] = categories
    output_row["task"] = tasks
    output_row["subtask"] = subtasks
    output_row = dict(output_row)
    output.append(output_row)

    final_output += output
    print("output_size:", len(output), "final_output size:", len(final_output))

## AIHUB_Twitter

In [ ]:
dataset_name = "AIHUB_Twitter"
language = "kor"
encoding = "UTF-8"
root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

df = pd.read_excel(raw_dir+"트위터_대화시나리오DB_2000Set.xlsx", engine="openpyxl")
df = df.fillna("")

output = []
for row in df.values.tolist():
    utterances = []
    for utterance in row:
        if utterance == "": break
        utterances.append(utterance)
    speaker_ids = [i%2 for i in range(0, len(utterances))]
    
    output_row = OrderedDict()
    output_row["utterances"] = utterances
    output_row["speaker_ids"] = speaker_ids
    output_row = dict(output_row)
    output.append(output_row)

## AIHUB_EMOTION

In [ ]:
language = "kor"
encoding = "UTF-8"
extension = "xlsx"

dataset_name = "AIHUB_EMOTION"
root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

df = pd.DataFrame()
for file_name in os.listdir(raw_dir):
    if not file_name.endswith(extension): continue
    _df = pd.read_excel(raw_dir+file_name)
    df = pd.concat([df, _df], axis=0)
df = df.fillna("")
df = df.drop_duplicates()

output = []
domain = "emotional_dialog"
for row_idx, row in enumerate(df.values.tolist()):
    _, _, age_range, gender, category, healt_status, task, subtask, u1, s1, u2, s2, u3, s3 = row
    if gender == "여성": gender = "F"
    elif gender == "남성": gender = "M"
    else: gender = "O"
    speakers = {"0":"시스템", "1":"사용자"}
    utterances = [u1, s1, u2, s2, u3, s3]
    utterances = [utterance for utterance in utterances if utterance!=""]
    speaker_ids = [(i+1)%2 for i in range(0, len(utterances))]
    
    output_row = OrderedDict()
    output_row["speakers"] = speakers
    output_row["domain"] = domain
    output_row["utterances"] = utterances
    output_row["speaker_ids"] = speaker_ids
    output_row["category"] = category
    output_row["task"] = task
    output_row["subtask"] = subtask
    output_row = dict(output_row)
    output.append(output_row)

## AIHUB_SNS

In [123]:
language = "kor"
encoding = "UTF-8"
extension = "json"

dataset_name = "AIHUB_SNS"
root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

data = []
for file_name in os.listdir(raw_dir):
    if not file_name.endswith(extension): continue
    with open(raw_dir+file_name, "r", encoding=encoding) as fp:
        _data = json.load(fp)
        data += _data["data"]
    break

output = []
for row_idx, row in enumerate(data):
    domain = row["header"]["dialogueInfo"]["type"]
    category = row["header"]["dialogueInfo"]["topic"]
    speakers = OrderedDict()
    speaker_mapping = dict()
    for speaker_id, speaker in enumerate(row["header"]["participantsInfo"]):
        speaker_mapping[speaker["participantID"]] = speaker_id
        speakers[speaker_id] = OrderedDict()
        speakers[speaker_id]["id"] = speaker["participantID"]
        speakers[speaker_id]["sex"] = speaker["gender"]
        speakers[speaker_id]["age"] = speaker["age"]
        speakers[speaker_id]["residence"] = speaker["residentialProvince"]
        speakers[speaker_id] = dict(speakers[speaker_id])
    speakers = dict(speakers)
    summary = row["body"]["summary"]
    
    utterances = []
    speaker_ids = []
    datetime_list = []
    for _row in row["body"]["dialogue"]:
        utterance = _row["utterance"]
        utterances.append(utterance)
        speaker_id = speaker_mapping[_row["participantID"]]
        speaker_ids.append(speaker_id)
        _datetime = _row["date"] + " " + _row["time"]
        datetime_list.append(_datetime)
    
    output_row = OrderedDict()
    output_row["idx"] = row_idx
    output_row["speakers"] = speakers
    output_row["domain"] = domain
    output_row["utterances"] = utterances
    output_row["speaker_ids"] = speaker_ids
    output_row["datetime"] = datetime_list
    output_row["category"] = category
    output_row["summary"] = summary
    output_row = dict(output_row)
    output.append(output_row)

## KLUE_DST

In [ ]:
dataset_name = "KLUE_DST" # 국립국어원
language = "kor"
encoding = "UTF-8"
root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

file_name_list = [file_name for file_name in os.listdir(raw_dir) if file_name.endswith(".json") and file_name.startswith("wos")]

output = []
kor_regex = "[ㄱ-ㅣ가-힣]"

for file_name in file_name_list:
    with open(raw_dir+file_name, "r", encoding=encoding) as fp:
        data = json.load(fp)

    for dialog_idx, dialog_row in enumerate(data):
        dialog = dialog_row["dialogue"]

        output_row = OrderedDict()
        utterances = []
        speaker_ids = []
        category = []
        task = []
        subtask = []
        _speaker_info = dict()
        for row in dialog:
            speaker = row["role"]
            if speaker not in _speaker_info:
                _speaker_info[speaker] = len(_speaker_info)
            utterance = row["text"]
            utterances.append(utterance)
            speaker_id = _speaker_info[speaker]
            speaker_ids.append(speaker_id)

            if "state" not in row: continue
            state = row["state"]
            for _state in state:
                _state = _state.split("-")
                if len(_state) > 0: 
                    _category = _state[0]
                    category.append(_category)
                if len(_state) > 1:
                    _task = _state[1]
                    task.append(_task)
                if len(_state) > 2:
                    _subtask = _state[2]
                    if re.search(kor_regex, _subtask) is not None:
                        subtask.append(_subtask)
            category = trim_obj(category)
            task = trim_obj(task)
            subtask = trim_obj(subtask)

        speaker_info = {v:k for k,v in _speaker_info.items()}

        output_row["idx"] = dialog_idx
        output_row["speakers"] = speaker_info
        output_row["utterances"] = utterances
        output_row["speaker_ids"] = speaker_ids
        output_row["category"] = category
        output_row["task"] = task
        output_row["subtask"] = subtask
        output_row = dict(output_row)
        output.append(output_row)

## NIKL_2020

In [ ]:
dataset_name = "NIKL_2020"
language = "kor"
encoding = "UTF-8"
root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

file_name_list = [file_name for file_name in os.listdir(raw_dir) if file_name.endswith(".json")]
output = []
for idx, file_name in enumerate(file_name_list):
    with open(raw_dir+file_name, "r", encoding=encoding) as fp:
        data = json.load(fp)

    # date
    # date = datetime.strptime(data["document"][0]["metadata"]["date"], "%Y%M%d").strftime("%Y-%M-%d")
    _date = data["document"][0]["metadata"]["date"]
    date = "-".join([_date[0:4], _date[4:6], _date[6:8]])
    # tasks
    tasks = data["document"][0]["metadata"]["topic"]
    tasks = tasks.split(">")[-1]
    tasks = [task.strip() for task in tasks.split(",")]
    tasks = trim_obj(tasks)
    # speaker_info
    spkid2id = {speaker["id"]:_idx for _idx, speaker in enumerate(data["document"][0]["metadata"]["speaker"])}
    speaker_info = {spkid2id[speaker["id"]]:speaker for speaker in data["document"][0]["metadata"]["speaker"]}
    speaker_info = dict(OrderedDict(sorted(speaker_info.items())))
    # utterances & speaker_ids
    utterances = []
    speaker_ids = []
    elements = data["document"][0]["utterance"]
    prev_utterance = elements[0]["form"] # 철자 전사
    # utterance = elements[0]["original_form"] # 발음 전사
    prev_speaker_id = elements[0]["speaker_id"]
    for element_idx in range(1, len(elements)):
        cur_utterance = elements[element_idx]["form"] # 철자 전사
        # cur_utterance = elements[element_idx]["original_form"] # 발음 전사
        cur_speaker_id = elements[element_idx]["speaker_id"]

        if cur_speaker_id == prev_speaker_id:
            if prev_utterance.endswith("."): 
                utterances.append(prev_utterance)
                speaker_ids.append(prev_speaker_id)
                prev_utterance = cur_utterance
            else:
                prev_utterance = prev_utterance + cur_utterance
        else:
            for _prev_utterance in prev_utterance.split(". "):
                if _prev_utterance.strip() == "": continue
                if prev_utterance.strip().endswith(".") and not _prev_utterance.strip().endswith("."): 
                    _prev_utterance = _prev_utterance + "."
                utterances.append(_prev_utterance)
                speaker_ids.append(prev_speaker_id)
            # reset
            prev_utterance = cur_utterance
            prev_speaker_id = cur_speaker_id
    utterances.append(prev_utterance)
    speaker_ids.append(prev_speaker_id)
    speaker_ids = [spkid2id[speaker_id] for speaker_id in speaker_ids]


    output_row = OrderedDict()
    output_row["idx"] = idx
    output_row["document_id"] = data["document"][0]["id"]
    output_row["date"] = date
    output_row["task"] = tasks
    output_row["speaker_info"] = speaker_info
    output_row["setting"] = data["document"][0]["metadata"]["setting"]
    output_row["utterances"] = utterances
    output_row["speaker_ids"] = speaker_ids
    output_row = dict(output_row)
    output.append(output_row)

## NIKL_Dialog

In [293]:
def get_output_row(idx, soup):
    output_row = OrderedDict()
    speaker_info = dict()
    _speaker_id_mapping = dict()
    utterances = []
    speaker_ids = []

    header = soup.find("body").find("teiheader")
    _file_desc = header.find("filedesc")
    _profile_desc = header.find("profiledesc")
    speakers = _profile_desc.findAll("person")
    for speaker_id, speaker in enumerate(speakers):
        _speaker_info = OrderedDict(speaker.attrs)
        if len(splited) > 0:
            _speaker_info["job"] = splited[0].strip()
        if len(splited) > 1:
            _speaker_info["residence"] = splited[1].strip()
        speaker_info[speaker_id] = dict(_speaker_info)
        _speaker_id_mapping[_speaker_info["id"]] = speaker_id

    utterances = []
    speaker_ids = []

    prev_speaker_id = -1
    prev_utterance = ""
    _utterances = soup.findAll("u")
    for _utterance in _utterances:
        _speaker_id = _utterance.attrs["who"]
        if _speaker_id not in _speaker_id_mapping: continue
        speaker_id = _speaker_id_mapping[_speaker_id]
        # utterance = [u.text.strip() for u in _utterance.findAll("s")]
        utterance = [child.strip() for s in _utterance.findAll("s") for child in s.children if isinstance(child, str)]
        utterance = " ".join(utterance)
        utterance = utterance.replace("::", "")
        if utterance == "": continue

        if speaker_id == prev_speaker_id:
            prev_utterance = prev_utterance + " " + utterance
        else:
            if prev_speaker_id != -1:
                utterances.append(prev_utterance)
                speaker_ids.append(prev_speaker_id)
            prev_utterance = utterance
            prev_speaker_id = speaker_id

    if speaker_id == prev_speaker_id:
        prev_utterance = prev_utterance + utterance
    utterances.append(prev_utterance)
    speaker_ids.append(prev_speaker_id)
    
    assert len(utterances) == len(speaker_ids), "{l1} vs {l2}".format(l1=len(utterances), l2=len(speaker_ids))

    output_row["idx"] = idx
    output_row["speakers"] = speaker_info
    output_row["title"] = _file_desc.find("title").text
    output_row["domain"] = _profile_desc.find("settingdesc").text
    output_row["utterances"] = utterances
    output_row["speaker_ids"] = speaker_ids
    output_row["project_name"] = header.find("projectdesc").text
    output_row["distributor"] = _file_desc.find("distributor").text
    extent = _file_desc.find("extent").text
    output_row["num_eojeols"] = int(re.sub("[^0-9]", "", extent))
    output_row = dict(output_row)
    return output_row

file_path_list = []
for sub_dir in os.listdir(raw_dir):
    file_path = raw_dir+sub_dir+"/원시/"
    if not os.path.isdir(file_path): continue
    file_name_list = os.listdir(file_path)
    if len(file_name_list) != 1: continue
    file_name = file_name_list[0]
    file_path_list.append(file_path+file_name)
    
output = []
for idx, file_path in enumerate(file_path_list):
    with open(file_path, 'rb') as f:
        contents = f.read()
    contents = contents.decode("utf-16")
    contents = re.sub("\r\n", "", contents)
    soup = BeautifulSoup(contents, 'lxml')
    output_row = get_output_row(idx=idx, soup=soup)
    output.append(output_row)

## OpenSubtitles

In [ ]:
dataset_name = "OpenSubtitles"
root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

pickle_path = file_path[dataset_name]["pickle"].format(root_dir=root_dir, language=language)
data = None
with open(pickle_path, "rb") as fp:
    data = pickle.load(fp)

output = []
kor_regex = "[ㄱ-ㅣ가-힣]"

_data = data["train"] + data["test"]
idx = 0
for row in _data:
    output_row = OrderedDict()
    output_row["idx"] = idx
    output_row["file_id"] = row.pop("file_id")
    context = row.pop("context")
    response = row.pop("response")
    
    _utterances = [row[k] for k in sorted(row.keys(), reverse=True)]
    _utterances.append(context)
    _utterances.append(response)

    utterances = []
    speaker_ids = []
    _idx = 0
    
    not_korean = True
    for utterance in _utterances:
        speaker_id = -1 * ((_idx+1) % 2)
        _idx += 1
        utterance = utterance.replace("'", "")
        utterance = utterance.replace('"', '')
        utterance = re.sub("^.* :", "", utterance).strip()
        utterance = utterance.replace("nbsp;", "").strip()
        if utterance=="": continue
        utterances.append(utterance)
        speaker_ids.append(speaker_id)
        if re.search(kor_regex, utterance) is not None: not_korean = False
    
    if not_korean: continue
        
    output_row["utterances"] = utterances
    output_row["speaker_ids"] = speaker_ids
    output_row = dict(output_row)
    output.append(output_row)
    idx += 1

## EmpatheticDialogues

#### kor

In [69]:
dataset_name = "SelectStar"
language = "kor"
encoding = "UTF-8"
extension = "json"

root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

data = []
for filename in os.listdir(raw_dir):
    if not filename.endswith(extension): continue
    with open(raw_dir+filename, "r", encoding=encoding) as fp:
        __data = json.load(fp)
        
        _data = []
        for row in __data:
            if "speakers_ids" in row:
                row["speaker_ids"] = row.pop("speakers_ids")
            _data.append(row)
        data += _data
print("data size:", len(data))
total_personas = [persona for row in data for persona in row["persona"].values()]
persona_df = pd.DataFrame(total_personas)
persona_df = persona_df.fillna("")

pd.options.mode.chained_assignment = None  # default='warn'
def find_most_similar_persona(age, gender, default_name="당신"):
    target_df = persona_df
    target_persona = {"name": default_name}
    try:
        target_df = target_df[target_df["gender"]==gender]
        age_diff = abs(target_df["age"] - age)
        target_df["age_diff"] = list(age_diff)
        target_age_diff = target_df.sort_values("age_diff").values.tolist()[0][-1]
        target_df = target_df.loc[(target_df["age_diff"]==target_age_diff)]
        target_persona = target_df.sample(1).to_dict("records")[0]
    except:
        target_persona = target_df.sample(1).to_dict("records")[0]
    return target_persona

data size: 2342


In [70]:
dataset_name = "EmpatheticDialogues"
language = "kor"
encoding = "UTF-8"
extension = "xlsx"
root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

filename_list = os.listdir(raw_dir)
df = pd.DataFrame()
for filename in filename_list:
    if not filename.endswith(extension): continue
    _df = pd.read_excel(raw_dir + filename)
    df = pd.concat([df, _df], axis=0)
df = df.fillna("")

In [71]:
# make preprocessed raw_data
data = []

name_entity_pattern = "\$\$[ㄱ-ㅣ가-힣]+\$\$"
prev_row_idx = -1
utterances = []
speaker_ids = []
personas = []
entities = dict()
turn_idx = 0

for _idx, row in enumerate(df.values.tolist()):    
    row_idx, persona_idx, eng, kor_before, kor_after, gender, age, married, job = row
    
    if _idx == 0: 
        prev_row_idx = row_idx
    if _idx > 0 and row_idx != "" and prev_row_idx != int(row_idx):
        # append dialog
        output_row = OrderedDict()
        output_row["idx"] = int(prev_row_idx)
        output_row["utterances"] = utterances
        output_row["speaker_ids"] = speaker_ids
        output_row["persona"] = personas
        output_row["entities"] = entities
        output_row = dict(output_row)
        data.append(output_row)
        # reset
        prev_row_idx = row_idx
        utterances = []
        speaker_ids = []
        personas = []
        entities = dict()
        turn_idx = 0
        
    _utterances = clean(str(kor_after))
    speaker_id = (turn_idx+1) % 2
    for utterance in _utterances.split(". "):
        utterances.append(utterance)
        speaker_ids.append(speaker_id)
        
    if gender!="":
        persona = OrderedDict()
        persona["id"] = speaker_id
        persona["persona_idx"] = int(persona_idx)
        persona["gender"] = gender
        if age != "": 
            persona["age"] = int(age)
        if married != "": 
            persona["married"] = int(married)
        if job != "": 
            persona["job"] = int(job)
        persona = dict(persona)
        personas.append(persona)
    turn_idx += 1
    
# append dialog
output_row = OrderedDict()
output_row["idx"] = int(prev_row_idx)
output_row["utterances"] = utterances
output_row["speaker_ids"] = speaker_ids
output_row["persona"] = personas
output_row["entities"] = entities
output_row = dict(output_row)
data.append(output_row)

In [72]:
# 이름 채워넣기
_data = data.copy()
data = []
for row in _data: 
    new_personas = []
    for persona in row["persona"]:
        new_persona = find_most_similar_persona(age=persona["age"], gender=persona["gender"])
        persona["name"] = new_persona["name"]
        new_personas.append(persona)
    row["persona"] = new_personas
    data.append(row)

In [73]:
# 이름 치환 & entities 정보 수집
name_entity_pattern = "\$\$[ㄱ-ㅣ가-힣]+\$\$"

_data = data.copy()
data = []

for _row in _data:
    row = _row.copy()
    
    utterances = []
    speaker_ids = []
    labels = []
    entities = dict()
    for utterance_idx, (_utterance, _speaker_id) in enumerate(zip(row["utterances"], row["speaker_ids"])):
        label = "" if _speaker_id == 1 else "N"
        
        while True:
            name_entity_search = re.search(name_entity_pattern, _utterance)
            if name_entity_search is not None:
                cur_persona = [persona for persona in row["persona"] if persona["id"]==(_speaker_id+1)%2][0]
                entity_start_idx = name_entity_search.start()
                entity_end_idx = name_entity_search.end()

                name = cur_persona["name"]
                if len(name) == 3: name = name[1:]
                _utterance = _utterance[:entity_start_idx] + name + _utterance[entity_end_idx:]
                entity_span = len(name)
                entity_end_idx = entity_start_idx + entity_span

                entity = [utterance_idx, entity_start_idx, entity_end_idx, entity_span]
                if "name" not in entities: entities["name"] = []
                entities["name"].append(entity)
                if _speaker_id == 0: label = "U"
            else:
                break

        utterances.append(_utterance)
        speaker_ids.append(_speaker_id)
        labels.append(label)

    row["utterances"] = utterances
    row["speaker_ids"] = speaker_ids
    row["labels"] = labels
    row["entities"] = entities
    
    data.append(row)

# with open(raw_dir + "empathetic_dialogues.pickle", "wb") as fp:
#     pickle.dump(data, fp)
# with open(root_dir + "/{language}/multi_turn/feed_data_0.json".format(language=language), "w", encoding=encoding) as fp:
#     json.dump(data, fp)

#### eng

In [ ]:
dataset_name = "EmpatheticDialogues"
language = "eng"
encoding = "UTF-8"
root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

_data = None
with open(raw_dir+"empathetic_dialogues.pickle", "rb") as fp:
    _data = pickle.load(fp)

output = []
row_idx = 0
for k,v in _data.items():
    print("v size:", len(v))
    print("v conv size:", len(pd.DataFrame(v)["conv_id"].unique()))
    rows = []

    prev_conv_id = None
    output_row = OrderedDict()
    for row in v:
        cur_conv_id = row["conv_id"]
        if prev_conv_id is None:
            output_row["idx"] = row_idx
            output_row["context"] = row["context"]
            output_row["prompt"] = row["prompt"]
            output_row["utterances"] = []
            output_row["speaker_ids"] = []
            output_row["selfeval"] = row["selfeval"]
            output_row["tags"] = []
            prev_conv_id = cur_conv_id
            continue

        if cur_conv_id != prev_conv_id:
            tags = sorted(set(output_row["tags"]))
            output_row["tags"] = tags
            prompt = output_row.pop("prompt")
            output_row["utterances"].insert(0, prompt)
            if len(output_row["speaker_ids"]) == 0:
                previous_speaker_id = 0
            elif len(output_row["speaker_ids"]) > 1:
                previous_speaker_id = output_row["speaker_ids"][1]
            else:
                if output_row["speaker_ids"][0] == 0:
                    previous_speaker_id = 1
                else:
                    previous_speaker_id = 0
            output_row["speaker_ids"].insert(0, previous_speaker_id)
            output_row = dict(output_row)
            rows.append(output_row)
            prev_conv_id = cur_conv_id
            row_idx += 1

            output_row = OrderedDict()
            output_row["idx"] = row_idx
            output_row["context"] = row["context"]
            output_row["prompt"] = row["prompt"]
            output_row["utterances"] = []
            output_row["speaker_ids"] = []
            output_row["selfeval"] = row["selfeval"]
            output_row["tags"] = []
        else:
            output_row["utterances"].append(row["utterance"])
            output_row["speaker_ids"].append(int(row["speaker_idx"]))
            output_row["tags"].append(row["tags"])

    tags = sorted(set(output_row["tags"]))
    output_row["tags"] = tags
    prompt = output_row.pop("prompt")
    output_row["utterances"].insert(0, prompt)
    if len(output_row["speaker_ids"]) > 1:
        previous_speaker_id = output_row["speaker_ids"][1]
    else:
        if output_row["speaker_ids"][0] == 0:
            previous_speaker_id = 1
        else:
            previous_speaker_id = 0
    output_row["speaker_ids"].insert(0, previous_speaker_id)
    output_row = dict(output_row)
    rows.append(output_row)
    prev_conv_id = cur_conv_id
    row_idx += 1
    
    print("rows size:", len(rows), "\n")
    output += rows

## KaggleConversation

In [ ]:
dataset_name = "KaggleConversation"
language = "kor"
encoding = "UTF-8"

root_dir = file_path[dataset_name]["root_dir"].format(dataset_dir=dataset_dir)
raw_dir = file_path[dataset_name]["raw_dir"].format(root_dir=root_dir, language=language)

titles = pd.read_csv(raw_dir + "conversation_titles.csv")
data_df = pd.read_csv(raw_dir + "conversations.csv")

output = []
for row_idx, _date in enumerate(titles["date"].unique().tolist()):
    title = titles[titles["date"]==_date]["kor_title"].tolist()[0]
    translated_title = titles[titles["date"]==_date]["eng_title"].tolist()[0]
    utterances = data_df[data_df["date"]==_date]["kor_sent"].tolist()
    translated_utterances = data_df[data_df["date"]==_date]["eng_sent"].tolist()
    speaker_ids = [(i+1)%2 for i in range(0, len(utterances))]
    
    output_row = OrderedDict()
    output_row["idx"] = row_idx
    output_row["title"] = title
    output_row["utterances"] = utterances
    output_row["speaker_ids"] = speaker_ids
    output_row["translated_title"] = translated_title
    output_row["translated_utterances"] = translated_utterances
    output_row = dict(output_row)
    output.append(output_row)